#### Triangle Inequality sketch for rank-one updates

Of course, there is no triangle inequality for eigenvalues. The eigensystem can be tracked over multiple updates using the trace, but that is a strict equality of course.

In [2]:
import numpy as np
np.set_printoptions(linewidth = 150, precision = 4, suppress = True)

In [3]:
data = np.loadtxt('../NC-Data.csv', delimiter=',', dtype=str)
data = data[1:].astype(float)
data

array([[ 1.    , -0.0461,  0.2312, ...,  0.2704,  0.4664,  0.3672],
       [-0.0461,  1.    , -0.0671, ..., -0.0515, -0.0944, -0.0349],
       [ 0.2312, -0.0671,  1.    , ...,  0.147 ,  0.2608,  0.6313],
       ...,
       [ 0.2704, -0.0515,  0.147 , ...,  1.    ,  0.2066,  0.1538],
       [ 0.4664, -0.0944,  0.2608, ...,  0.2066,  1.    ,  0.3486],
       [ 0.3672, -0.0349,  0.6313, ...,  0.1538,  0.3486,  1.    ]])

In [4]:
cov = data[:3, :3]
cov

array([[ 1.    , -0.0461,  0.2312],
       [-0.0461,  1.    , -0.0671],
       [ 0.2312, -0.0671,  1.    ]])

In [5]:
d1, d2 = np.random.choice(np.arange(3, len(data)), 2, replace=0)
# Include a choice here which breaks.
d1, d2 = 85, 91
d1, d2

(85, 91)

In [6]:
eigvals, eigvecs = np.linalg.eigh(cov)
eigvals

array([0.7678, 0.9759, 1.2563])

In [10]:
proj_updates = eigvecs.T @ data[0:3, [d1, d2]] / eigvals[:, None]
proj_updates = np.r_[
    proj_updates,
    np.sqrt(data[[d1, d2], [d1, d2]][None, :]
        - np.linalg.norm(proj_updates, axis=0, keepdims=1) ** 2),
]
proj_updates

array([[0.2212, 0.2206],
       [0.1236, 0.333 ],
       [0.2077, 0.0353],
       [0.9448, 0.9161]])

In [11]:
# Trivial upper bound on dominant eigval, from the trace.
eigvals[-1] + data[d1, d1] + data[d2, d2]

3.2562505988455444

In [12]:
# Actual dominant eigenvalue.
np.linalg.eigvalsh(data[[0, 1, 2, d1, d2], :][:, [0, 1, 2, d1, d2]])[-1]

1.7439990337198588

In [13]:
def resize(arr, size):
    # Pad tuple: zeroes inserted before, and zeroes inserted after.
    return np.pad(arr, [(0, new_len - arr.shape[i]) for i, new_len in enumerate(size)])

In [17]:
eig_updates = [
    np.linalg.eigvalsh(
        resize(cov, [4, 4]) + proj_updates[:, i:i+1] @ proj_updates[:, i:i+1].T
    )[-1]
    - eigvals[-1]
    for i in range(proj_updates.shape[1])
]
eig_updates

[0.2105508373465872, 0.1396109793096172]

In [18]:
eigvals[-1] + sum(eig_updates)

1.606412415501749